<a href="https://colab.research.google.com/github/ImTeddyGraham/CompSci-ClassProj/blob/main/WIP_Audio_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [119]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [118]:
import pandas as pd
import matplotlib.pyplot as plt
import math, random
import torch
import torchaudio
from IPython.display import Audio

In [8]:
! mkdir ~/.kaggle

In [11]:
! cp kaggle.json ~/.kaggle/

In [12]:
! chmod 600 ~/.kaggle/kaggle.json

In [13]:
! kaggle datasets download -d chrisfilo/urbansound8k

100% 5.60G/5.61G [00:44<00:00, 171MB/s]
100% 5.61G/5.61G [00:44<00:00, 135MB/s]


In [115]:
data_info = pd.read_csv('/content/UrbanSound8K.csv')
data_info['clip_path'] = '/fold' + data_info['fold'].astype(str) + '/' + data_info['slice_file_name'].astype(str)
dataset = data_info[['clip_path', 'classID']]

In [124]:
dataset.head()

,clip_path,classID
0,/fold5/100032-3-0-0.wav,3
1,/fold5/100263-2-0-117.wav,2
2,/fold5/100263-2-0-121.wav,2
3,/fold5/100263-2-0-126.wav,2
4,/fold5/100263-2-0-137.wav,2


In [70]:
data_info.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class,relative_path
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark,/fold5/100032-3-0-0.wav
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing,/fold5/100263-2-0-117.wav
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing,/fold5/100263-2-0-121.wav
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing,/fold5/100263-2-0-126.wav
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing,/fold5/100263-2-0-137.wav


In [121]:
test_clip = '/content/fold1/101415-3-0-2.wav'

In [122]:
class AudioTools():
  def audio_info(audio_clip):
    sig, sr = torchaudio.load(audio_clip)
    return (sig, sr)

  def channel_converter(audio, channels=2):
    sig = audio[0]
    
    if (sig.shape[0] == channels):
      # if clip has desired number of channels do nothing
      pass
    elif (channels == 2):
      # deletes second channel STEREO -> MONO  
      sig = sig[:1,:]
    else:
      # duplicates first channel MONO -> STEREO
      sig = torch.cat([sig, sig])
    
    return (sig, audio[1])

  def standardize_len(audio, max_ms):
    sig, sr = audio
    sig_channels, sig_len = sig.shape

    max_len = sr//1000 * max_ms

    if (sig_len > max_len):
      # clip is too long and must be truncated
      sig = sig[:,:max_len]

    elif (sig_len < max_len):
      # randomly pads start and end of clip 
      start_pad_size = random.randint(0, max_len - sig_len)
      end_pad_size = max_len - start_pad_size
      # applies padding 
      start_pad = torch.zeros((sig_channels, start_pad_size))
      end_pad = torch.zeros((sig_channels, end_pad_size))
      
      # updates padded audio clip 
      sig = torch.cat((start_pad_size, sig, end_pad_size), 2)

    return sig

In [123]:
signal, sample_rate = torchaudio.load(test_clip)